# Segmenting Clustering Assignment
## Requirements
For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.
Start by creating a new Notebook for this assignment. Use the Notebook to build the code to scrape the 
following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:





## PREPROCESSING

In [1]:
# importing necessary libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize 



In [2]:
# getting data from internet
# using beautiful soup to parse the HTML/XML codes.
# https://pypi.python.org/pypi/beautifulsoup4
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import csv

print('BeautifulSoup  & csv imported.')



BeautifulSoup  & csv imported.


In [3]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'lxml')
#print(soup.prettify()
print('soup ready')

soup ready


In [4]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'lxml')

table = soup.find("table")
table_rows = table.tbody.find_all("tr")

res = []
for tr in table_rows:
    td = tr.find_all("td")
    row = [tr.text for tr in td]
    
    # Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    if row != [] and row[1] != "Not assigned":
        # If a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough.
        if "Not assigned" in row[2]: 
            row[2] = row[1]
        res.append(row)

# Dataframe with 3 columns
df = pd.DataFrame(res, columns = ["PostalCode", "Borough", "Neighborhood"])
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows
#df.head()

In [5]:
df["Neighborhood"] = df["Neighborhood"].str.replace("\n","")
#df.head()

In [6]:
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
#df.head()

In [7]:
df1 = df.reset_index()
#df1.head()

In [8]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 4 columns):
index           180 non-null int64
PostalCode      180 non-null object
Borough         180 non-null object
Neighborhood    180 non-null object
dtypes: int64(1), object(3)
memory usage: 5.7+ KB


In [9]:
df1.shape

(180, 4)

More than one neighborhood can exist in one postal code area, M5A is listed twice and has two neighborhoods Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma using groupby, see:

https://pandas-docs.github.io/pandas-docs-travis/user_guide/groupby.html

In [10]:
df2= df1.groupby('PostalCode').agg(lambda x: ','.join(x))

#df2.head()
       

In [12]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 180 entries, M1A
 to M9Z

Data columns (total 2 columns):
Borough         180 non-null object
Neighborhood    180 non-null object
dtypes: object(2)
memory usage: 4.2+ KB


In [23]:
df2["Neighborhood"] = df["Neighborhood"].str.replace("\n","")
#df2.head()

In [36]:
df["Borough"] = df["Borough"].str.replace("\n","")
#df.head()

In [27]:
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
2,M3A\n,North York,Parkwoods
3,M4A\n,North York,Victoria Village
4,M5A\n,Downtown Toronto,Regent Park / Harbourfront
5,M6A\n,North York,Lawrence Manor / Lawrence Heights
6,M7A\n,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [29]:
df["PostalCode"] = df["PostalCode"].str.replace("\n","")
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


There are also cells that have an assigned neighbouhoods,like M7A, lets assign their boroughs as their neighbourhood, as follows:

In [30]:
df2 = df.groupby(["PostalCode", "Borough"])["Neighborhood"].apply(", ".join).reset_index()
df2.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [31]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
PostalCode      103 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
dtypes: object(3)
memory usage: 2.5+ KB


In [32]:
df2.shape

(103, 3)

In [33]:
df2.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
df2.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [37]:
df2 = df.groupby(["PostalCode", "Borough"])["Neighborhood"].apply(", ".join).reset_index()
df2 = df[~df['PostalCode'].isnull()]  # to filter out bad rows
df2.head(14)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge
11,M3B,North York,Don Mills
12,M4B,East York,Parkview Hill / Woodbine Gardens
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [38]:
df2.shape

(103, 3)